In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

print("🎞️ Please upload a short video file (MP4/AVI, under 10–20 MB recommended):")
uploaded = files.upload()

# Load uploaded video
for fn in uploaded.keys():
    video_path = fn

# Open video
cap = cv2.VideoCapture(video_path)
ret, prev = cap.read()
if not ret:
    raise ValueError("Could not read the video. Please upload a valid one.")

prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(prev)
hsv[..., 1] = 255

frame_idx = 0
snapshots = []  # store a few motion maps

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None,
                                        pyr_scale=0.5, levels=3, winsize=15,
                                        iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    # Save every 20th frame for display
    if frame_idx % 20 == 0:
        snapshots.append((frame.copy(), rgb.copy()))
    prev_gray = gray

cap.release()

# Display few sample frames with motion
print(f"✅ Processed video — showing {len(snapshots)} key motion frames")

cols = 2
rows = len(snapshots)
plt.figure(figsize=(12, 4 * rows))

for i, (orig, motion) in enumerate(snapshots):
    plt.subplot(rows, cols, 2*i + 1)
    plt.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
    plt.title(f"Original Frame #{i*20}")
    plt.axis('off')

    plt.subplot(rows, cols, 2*i + 2)
    plt.imshow(motion)
    plt.title(f"Motion Visualization #{i*20}")
    plt.axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.